In [19]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [20]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [21]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [22]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data\\ravdessdata\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]

        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=0)

In [23]:
# Split the dataset
X_train,X_test,y_train,y_test=load_data(test_size=0.20)

In [24]:
# Get the shape of the training and testing datasets
print((X_train.shape[0], X_test.shape[0]))

(1152, 288)


In [25]:
# Get the number of features extracted
print(f'Features extracted: {X_train.shape[1]}')

Features extracted: 180


## Create a Logistic Regression Model

In [27]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [28]:
classifier.fit(X_train, y_train)

C:\Users\gucas\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [29]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.4809027777777778
Testing Data Score: 0.3993055555555556


In [35]:
# y_pred = classifier.predict(X_test)
# y_pred

In [31]:
pd.DataFrame({'Prediction': y_pred, 'Actual': y_test}).head(60)

,Prediction,Actual
0,angry,angry
1,fearful,fearful
2,sad,surprised
3,angry,angry
4,calm,sad
5,calm,calm
6,sad,calm
7,disgust,fearful
8,sad,disgust
9,disgust,disgust


In [12]:
# model.predict(x_test[:1])

In [13]:
# x_test[0]

In [32]:
#----------------------------#
#    Personal Audio test     #
#----------------------------#
feature22=extract_feature('data/kg.wav', mfcc=True, chroma=True, mel=True)
# feature22
# ValueError: Expected 2D array, got 1D array instead:
# array=[-326.22702026].
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
kg_predit = classifier.predict(feature22.reshape(1, -1))
kg_predit

array(['angry'], dtype='<U9')

In [33]:
kg_predit[0]

'angry'

In [34]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 39.93%


In [ ]:
# feature2=extract_feature('data/ravdessdata/Actor_16/03-01-01-01-01-01-16.wav', mfcc=True, chroma=True, mel=True)
# feature2